In [75]:
import pandas as pd
import string
import re
import nltk
import numpy as np
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import gensim
import itertools
from math import isnan

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/utkarsh.verma/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/utkarsh.verma/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Preparing Data


In [76]:
# Data for the model
articles = pd.read_csv (r'archiveNLP/shared_articles.csv')
users = pd.read_csv (r'archiveNLP/users_interactions.csv')

# combining the articles and users on contentid
combined = pd.merge(articles, users,how='left', on='contentId')
#combined = combined.iloc[1:1000,]

In [77]:
# Creating Dictionary 
dict_article1 = dict(zip(combined.title , combined.text))
dict_article=dict(itertools.islice(dict_article1.items(),500))

In [78]:
                            ### Preprocessing the data

import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
def preprocess(sentence):
    ps = WordNetLemmatizer()
    corpus = []
    #pattern = '[0-9]'
    review_text = re.sub('[^A-Za-z]+', ' ', sentence)
    review_text = review_text.lower()
    review_text = review_text.split()
    review_text = [ps.lemmatize(word) for word in review_text if not word in set(stopwords.words('english'))]
    #review_text = ' '.join(review_text)
    #corpus.append(review_text)
    return review_text

### Loading pre-trained Word2vec Model

In [79]:
# Load Google's pre-trained Word2Vec model.

model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [80]:
model.init_sims(replace=True)

In [81]:
# Generating User2Title Dictionary

usertitle = combined[['title','personId']].groupby("personId")['title'].apply(lambda x: x.tolist()).to_dict()

# Generating User2Title Dictionary

title2text = {k:preprocess(v) for (k, v) in dict_article.items()}

In [82]:
### Test preprocess function

sentence=dict_article["Ethereum, a Virtual Currency, Enables Transactions That Rival Bitcoin's"]
preprocess(sentence)

['work',
 'still',
 'early',
 'first',
 'full',
 'public',
 'version',
 'ethereum',
 'software',
 'recently',
 'released',
 'system',
 'could',
 'face',
 'technical',
 'legal',
 'problem',
 'tarnished',
 'bitcoin',
 'many',
 'bitcoin',
 'advocate',
 'say',
 'ethereum',
 'face',
 'security',
 'problem',
 'bitcoin',
 'greater',
 'complexity',
 'software',
 'thus',
 'far',
 'ethereum',
 'faced',
 'much',
 'le',
 'testing',
 'many',
 'fewer',
 'attack',
 'bitcoin',
 'novel',
 'design',
 'ethereum',
 'may',
 'also',
 'invite',
 'intense',
 'scrutiny',
 'authority',
 'given',
 'potentially',
 'fraudulent',
 'contract',
 'like',
 'ponzi',
 'scheme',
 'written',
 'directly',
 'ethereum',
 'system',
 'sophisticated',
 'capability',
 'system',
 'made',
 'fascinating',
 'executive',
 'corporate',
 'america',
 'ibm',
 'said',
 'last',
 'year',
 'experimenting',
 'ethereum',
 'way',
 'control',
 'real',
 'world',
 'object',
 'called',
 'internet',
 'thing',
 'microsoft',
 'working',
 'several',
 'p

In [83]:
def titlevec_cal(first_map,second_map):
    temp = list()
    new_vec = dict()
    for (k, v) in first_map.items():
        for w in v:
            if w in second_map.vocab and len(w)<20:
                temp.append(second_map[w])
        new_vec[k]= np.mean(temp,axis=0)    
    return new_vec 

In [1]:
def uservec_cal(first_map,second_map):
    temp = list()
    new_vec = dict()
    for (k, v) in first_map.items():
        for w in v:
            if w in second_map.keys() :
                temp.append(second_map[w])
        new_vec[k]= np.mean(temp,axis=0)    
    return new_vec 

In [85]:
titlevec=titlevec_cal(title2text,model)
Uservec=uservec_cal(usertitle,titlevec)


In [86]:
# Removing the nan value from dictionary

clean_dict = {k:[elem for elem in v if elem is not np.nan] for k,v in Uservec.items()}

In [87]:
# function to calculate Similar users

from sklearn.metrics.pairwise import cosine_similarity
def similar_users(user_vec,users):
    sim=list()
    for i,v in users.items():
        sim.append((i,cosine_similarity(user_vec.reshape(1, -1), v.reshape(1, -1))[0][0]) )
    return sim    

In [88]:
x=similar_users(Uservec[-9.223121837663644e+18],Uservec)


### Users & Articles Similarity

In [111]:
similar_user_ids=[key[0] for key in sorted(x)[:5]]
rec_title=[usertitle[x] for x in similar_user_ids]

flat_list = [item for sublist in rec_movies for item in sublist]
title_filter=usertitle[-9.223121837663644e+18]
filtered_rec_title=[x for x in flat_list  if not x in movie_filter ]
set(filtered_rec_title)

{'15 minutos sobre Docker',
 'A minha viagem à Maternidade #tetodomundo',
 'Adobe, Acquia, Episerver Lead New Forrester WCM Wave',
 'Aposta na inovação',
 'Cognizant Named a Digital Transformation Leader in the Life Science Industry',
 'Como são escrita as risadas em japonês? - Suki Desu',
 'Contêiner do Windows no Windows 10',
 'Docker - Build, Ship, and Run Any App, Anywhere',
 'Firebase Test Lab for Android',
 'Firebase and Google Cloud: better together',
 'Former Google career coach shares a visual trick for figuring out what to do with your life',
 'Fresco, sim! - Android Dev BR',
 'Gartner Hype Cycle for Digital Marketing and Advertising',
 'How to Create an Exponential Mindset',
 'Intel buys computer vision startup Itseez to improve navigation in self-driving cars',
 'Itaú Unibanco fecha compra de ativos de varejo do Citi no Brasil',
 'Itaú é pioneiro do blockchain',
 'Japanese for dummies',
 'Lançamento da LBI Comentada',
 'Learn Hiragana: The Ultimate Guide',
 'Request lesson 